In [1]:
import numpy as np
import matplotlib.pylab as plt
import scipy.sparse as sparse
from scipy.stats import truncnorm 
import pickle
import pandas as pd

In [2]:
m = 58415 # hyperedge = product = 58415
n = 849185 # node = basket = 849185

In [3]:
labels = [506] * n

In [4]:
train_order_no = []
with open("task1_train_label.txt", "r") as f:
    while True:
        line = f.readline()
        if not line: # 파일 읽기가 종료된 경우
            break
        order, return_label = line.strip().split('\t')
        train_order_no.append(int(order))
        labels[int(order)] = int(return_label)

In [5]:
valid_order_no = []
with open("task1_valid_label.txt", "r") as f:
    while True:
        line = f.readline()
        if not line: # 파일 읽기가 종료된 경우
            break
        order, return_label = line.strip().split('\t')
        valid_order_no.append(int(order))
        labels[int(order)] = int(return_label)

In [6]:
test_order_no = []
with open("task1_test_query.txt", "r") as f:
    while True:
        line = f.readline()
        if not line: # 파일 읽기가 종료된 경우
            break
        order = line.strip()
        test_order_no.append(int(order))
        labels[int(order)] = -1

In [7]:
# We split the dataset into a training set (594,430 orders),
# a  validation set (127,377 orders), 
# and a test set (127,378 orders), following a 70/15/15 distr


assert len(train_order_no) == 594430
assert len(valid_order_no) == 127377
assert len(test_order_no) == 127378
assert sum(np.array(labels) == -1) == 127378
assert sum(np.array(labels) == 506) == 0
assert len(train_order_no) + len(valid_order_no) + len(test_order_no) == n

In [8]:
with open("etail/ours/labels.pickle", 'wb') as f:
    pickle.dump(labels, f)


with open("etail/ours/train_labels.pickle", 'wb') as f:
    pickle.dump(train_order_no, f)

with open("etail/ours/valid_labels.pickle", 'wb') as f:
    pickle.dump(valid_order_no, f)

with open("etail/ours/test_labels.pickle", 'wb') as f:
    pickle.dump(test_order_no, f)

In [9]:
from collections import defaultdict

hypergraph = defaultdict(list)

with open("../data/task1_data.txt", "r") as f:
    f.readline()
    while True:
        line = f.readline()
        if not line: # 파일 읽기가 종료된 경우
            break
        order, product, customer, color, size, group = line.strip().split(',')
        hypergraph[int(product)].append(int(order))


In [10]:
assert len(set(hypergraph.keys())) == m

In [11]:
with open("etail/ours/hypergraph.pickle", 'wb') as f:
    pickle.dump(hypergraph, f)

In [22]:
from gensim.models import Word2Vec

df = pd.read_csv("task1_data.txt")

# 사용자별 아이템 시퀀스 생성
product_sequences = df.groupby('customer')['product'].apply(list).tolist()
max_len = max([len(seq) for seq in product_sequences])
print(max_len)

# Word2Vec 모델 학습
model = Word2Vec(sentences=product_sequences, vector_size=50, window=max_len, min_count=1, workers=32)


keys = [i for i in range(len(model.wv.index_to_key))]
vectors = model.wv[keys]

import scipy.sparse as sp
matrix = sp.csr_matrix(vectors)

1406


In [23]:
assert matrix.shape[0] == m

In [24]:
with open("etail/ours/features.pickle", 'wb') as f:
    pickle.dump(matrix, f)